# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
file_path = "../output_data/cities.csv"
cities_data_df = pd.read_csv(file_path, sep=",")
cities_data_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#configure gmaps
gmaps.configure(api_key= g_key)

In [ ]:
#Use the Lat and Lng as locations and Humidity as the weight.

location = cities_data_df[['Lat', 'Lng']]
humidity = cities_data_df['Humidity']
#max_humidity = cities_data_df['Humidity'].max()

fig = gmaps.figure()
#Add Heatmap layer to map.
fig.add_layer(gmaps.heatmap_layer(location, weights=humidity))
#fig


### Create new DataFrame fitting weather criteria
* Narrow down the Dataframe to find your ideal weather condition

* Drop any rows will null values.

In [ ]:
#Narrow down the Dataframe to find your ideal weather condition

#ideal weather 
ideal_weather_df = cities_data_df.loc[(cities_data_df['Max Temp']<80) & (cities_data_df['Max Temp']>70)]

#print ideal weather condition

ideal_weather_df
len(ideal_weather_df)

In [ ]:
# look for wind that is less than 10
ideal_wind_df = ideal_weather_df[(cities_data_df['Wind Speed']<10)]
len(ideal_wind_df)

In [ ]:
# zero cloudiness
ideal_cloud_df = ideal_wind_df.loc[(cities_data_df['Cloudiness']<2)]
len(ideal_cloud_df)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Store into variable named `hotel_df`.
hotel_df = ideal_cloud_df
#hotel_df
#place a right index
hotel_df.index = range(len(hotel_df))

#print hotel Dataframe
hotel_df

In [ ]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name']=""
hotel_df

In [ ]:
#Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = "lodging"

#base url
base_url = "http://maps.googleapis.com/maps/api/place/nearbysearch/json"

for item in hotel_df.index:
    target_coord = hotel_df['Lat'][item].astype(str) + "," + hotel_df['Lng'][item].astype(str)
    
    params = {
        "location": target_coord,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
              
    }

#get params
    response = requests.get(base_url, params=params)
    place_data = response.json()
    
    
    try:
        hotel_df['Hotel Name'][item] = place_data["results"][0]["name"]
    except:
        print(f"There are no hotels in this vincity for {hotel_df['City'][item]}.")
        hotel_df['Hotel Name'][item] = "None"
    
    

In [ ]:
#check to see hotel name

hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
